In [1]:
import torch
from transformers import BertTokenizer, BertForNextSentencePrediction, T5Tokenizer, T5ForConditionalGeneration
from transformers import BartTokenizer, BartForConditionalGeneration
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

In [2]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [3]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [4]:
text ="""Aam Aadmi Party (AAP) MLA and Delhi minister Atishi was picked by the party on Tuesday as Chief Minister Arvind Kejriwal’s successor, making the 43-year-old the youngest woman to occupy the post. Atishi will replace Kejriwal who, days after walking out on bail in the alleged excise policy scam, announced that he was stepping down.
A look at other women who have been CMs in various states across the country:
India’s first woman CM and the state’s fourth, Kripalani was 55 years old when she replaced Chandra Bhanu Gupta to head UP in 1963. She was the wife of Congress stalwart J B Kripalani and a minister in the Gupta Cabinet when the Congress picked her as the CM. She held the post for over three years. Sucheta was founder chief of All India Mahila Congress and parted ways with the Congress in 1969 when the party split, siding with Morarji Desai. After that, she contested the 1971 Lok Sabha polls as a candidate of the Congress (Organisation), but lost. she died in 1974. Both Kripalanis were Constituent Assembly members.
Satpathy was a Rajya Sabha member and a Union minister before returning to Odisha politics in 1972 after several top leaders left the Congress, including Biju Patnaik, and becoming the state’s first woman CM at the age of 41. She held office for just over four years. She split from the Congress later and was a part of Jagjivan Ram’s Congress for Democracy (CFD) in the post-Emergency 1977 elections, and won the Dhenkenal Assembly seat. The author of several short stories and poems, and the winner of Sahitya Bharati Sammana for the Odia translation of famous novel Lajja, Nandini later returned to the Congress and was a member of the Odisha Assembly till 2000 and died in 2006.
Shashikala Kakodkar, Goa, Daman & Diu served as a minister in the Maharashtrawadi Gomantak Party (MGP) government led by her father Dayanand Bandodkar in 1972 and became the first woman CM of the state at the age of 38 in 1973 after her father’s death.
Kakodkar was sworn in as CM for the second time in 1977 after leading the MGP to victory in 15 of the 30 seats and held the post until she resigned in 1979. In the 1980 Lok Sabha elections, Kakodkar successfully campaigned for MGP candidate Sanyogita Rane Sardesai for the North Goa seat, thus enabling the latter to become the first woman to be elected as MP from the state.
J Jayalalithaa, Tamil Nadu, The AIADMK stalwart, who first won an Assembly election in 1989, served as the Leader of the Opposition in the Tamil Nadu Assembly before assuming the CM’s chair two years later at the age of 41, becoming the youngest person to helm the state. In 1996, Jayalalithaa became the first woman CM of the state to serve a full term. She went on to become a five-time CM between 1991 and 2016, and passed away while in office.
Mayawati, Uttar Pradesh, Kanshi Ram’s successor, Mayawati first became CM in 1995 at the age of 39. She went on to serve as the CM of Uttar Pradesh four times. Her first three terms between 1995 and 1997, when she ran the government in alliance with other parties like the Samajwadi Party (SP) and BJP, did not last their entire tenures but Mayawati led the Bahujan Samaj Party (BSP) to majority in the 2007 Assembly elections and completed her term as CM. She is the first UP CM to complete a five-year tenure and is the second longest serving CM of the state, only after Yogi Adityanath.
"""

In [5]:
def preprocess_bert(text1, text2):
    encoding = bert_tokenizer.encode_plus(text1, text2, return_tensors='pt')
    return encoding['input_ids'], encoding['token_type_ids'], encoding['attention_mask']

def preprocess_summarization(text, tokenizer, model, max_length=150, min_length=40):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [6]:
def next_sentence_prediction(text1, text2):
    input_ids, token_type_ids, attention_mask = preprocess_bert(text1, text2)
    outputs = bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1)
    return probs

In [7]:
sentence1 = "Aam Aadmi Party MLA Atishi was picked by the party."
sentence2 = "She will replace Kejriwal, who stepped down recently."

In [8]:
probs = next_sentence_prediction(sentence1, sentence2)
print(f"Next sentence prediction probabilities: {probs}")

Next sentence prediction probabilities: tensor([[9.9998e-01, 1.5990e-05]], grad_fn=<SoftmaxBackward0>)


In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
def extractive_summarization(text, compression_ratio=0.6):
    sentences = sent_tokenize(text)
    num_sentences = int(len(sentences) * (1 - compression_ratio))

    sentence_embeddings = []
    for sentence in sentences:
        input_ids = bert_tokenizer.encode(sentence, return_tensors='pt')
        with torch.no_grad():
            outputs = bert_model.bert(input_ids)[1]
        sentence_embeddings.append(outputs.numpy())

    sentence_embeddings = np.vstack(sentence_embeddings)
    sentence_similarity = cosine_similarity(sentence_embeddings)

    ranked_sentences = np.argsort(np.mean(sentence_similarity, axis=1))[-num_sentences:]

    summary = " ".join([sentences[i] for i in sorted(ranked_sentences)])
    return summary

In [11]:
extractive_summary = extractive_summarization(text)
print("Extractive Summary:", extractive_summary)

Extractive Summary: Aam Aadmi Party (AAP) MLA and Delhi minister Atishi was picked by the party on Tuesday as Chief Minister Arvind Kejriwal’s successor, making the 43-year-old the youngest woman to occupy the post. Atishi will replace Kejriwal who, days after walking out on bail in the alleged excise policy scam, announced that he was stepping down. She was the wife of Congress stalwart J B Kripalani and a minister in the Gupta Cabinet when the Congress picked her as the CM. Kakodkar was sworn in as CM for the second time in 1977 after leading the MGP to victory in 15 of the 30 seats and held the post until she resigned in 1979. J Jayalalithaa, Tamil Nadu, The AIADMK stalwart, who first won an Assembly election in 1989, served as the Leader of the Opposition in the Tamil Nadu Assembly before assuming the CM’s chair two years later at the age of 41, becoming the youngest person to helm the state. In 1996, Jayalalithaa became the first woman CM of the state to serve a full term. She wen

In [12]:
t5_summary = preprocess_summarization(text, t5_tokenizer, t5_model)
print("T5 Abstractive Summary:", t5_summary)

T5 Abstractive Summary: atishi, 43, will replace chief minister arvind Kejriwal. she was the wife of stalwart J B Kripalani and a minister in the Gupta cabinet. she was the wife of the stalwart and a minister in the Gupta cabinet.


In [13]:
bart_summary = preprocess_summarization(text, bart_tokenizer, bart_model)
print("BART Abstractive Summary:", bart_summary)

BART Abstractive Summary: Aam Aadmi Party (AAP) MLA and Delhi minister Atishi was picked by the party on Tuesday as Chief Minister Arvind Kejriwal’s successor. Atishi will replace Kejriwal who, days after walking out on bail in the alleged excise policy scam, announced that he was stepping down.


In [15]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00


In [17]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [20]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=6f585b06cd3f5f7396aa000dc195f47aa4132826921ac35a872c3d60dd13185e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [21]:
import evaluate

rouge = evaluate.load("rouge")

In [23]:
reference = """Atishi was chosen as Delhi's youngest CM replacing Kejriwal after his resignation. Other notable women CMs include Sucheta Kripalani, Nandini Satpathy, Shashikala Kakodkar, Jayalalithaa, and Mayawati."""

In [24]:
def evaluate_summary(predicted_summary, reference_summary):
    scores = rouge.compute(predictions=[predicted_summary], references=[reference_summary])
    return scores

In [25]:
t5_rouge = evaluate_summary(t5_summary, reference)
bart_rouge = evaluate_summary(bart_summary, reference)

print("T5 ROUGE Scores:", t5_rouge)
print("BART ROUGE Scores:", bart_rouge)

T5 ROUGE Scores: {'rouge1': 0.15384615384615383, 'rouge2': 0.0, 'rougeL': 0.12307692307692308, 'rougeLsum': 0.12307692307692308}
BART ROUGE Scores: {'rouge1': 0.2191780821917808, 'rouge2': 0.028169014084507043, 'rougeL': 0.16438356164383558, 'rougeLsum': 0.16438356164383558}
